In [1]:
from numpy.ma.core import indices
from transformers import AutoTokenizer, AutoModelForCausalLM

llama_1b = AutoModelForCausalLM.from_pretrained('models/meta-llama/llama-3.2-1B')
llama_3b = AutoModelForCausalLM.from_pretrained('models/meta-llama/llama-3.2-3B')
tokenizer = AutoTokenizer.from_pretrained('tokenizers/meta-llama/llama-3.2-1B')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
import pickle
with open('data/chunked_wikitext2/train.pkl', 'rb') as f:
    train_set = pickle.load(f)

In [3]:
print(train_set[0])

 = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newc

In [4]:

#chunks by subsection. Found by preceding, ending = s.
def find_chunk(text:str) -> list:
    out = []
    text_stream = ""
    text_s = text.split("\n")
    for _ in text_s:
        _ = _.strip()
        if _.startswith("=") and _.endswith('='):
            out.append(text_stream)
            text_stream = _
        else:
            text_stream += "\n"
            text_stream += _
    return out



In [5]:
a = find_chunk(train_set[0])

In [6]:
len(a)

9

In [7]:
for _ in a:
    print(_[:20])
    print("------")


------
= Valkyria Chronicle
------
= = Gameplay = =
As 
------
= = Plot = =
The gam
------
= = Development = =

------
= = = Music = = =
Th
------
= = = Release = = =

------
= = Reception = =
On
------
= = Legacy = =
Kurt 
------


In [8]:
print(len(train_set))

629


In [9]:
chunked_by_sections = []
for i, text in enumerate(train_set):
    chunks = find_chunk(text)
    chunked_by_sections.extend(chunks)

In [10]:
len(chunked_by_sections)

6211

In [11]:
out = tokenizer(chunked_by_sections[1])['input_ids']

In [12]:
chunked_tokens = [tokenizer(_)['input_ids'] for _ in chunked_by_sections]

In [13]:
length_chunked = [len(_) for _ in chunked_tokens]
print(sum(length_chunked) / len(length_chunked))
print(max(length_chunked))

360.7155047496377
3067


In [14]:
import torch
ex_t = torch.tensor(chunked_tokens[1])
ex_t

tensor([128000,     28,  86262,     88,   4298,  66416,  14767,   4125,  20190,
            73,  56761,    912,  86262,     88,   4298,    220,     18,    551,
          1252,   8716,    291,  66416,    320,  11002,    551,  50534,     99,
         75267,  16144, 115687,  33710, 123283, 104612,     18,   1174,  13318,
           662,  86262,     88,   4298,    315,    279,  71735,    220,     18,
           883,   1174,  17037,  14183,    311,    439,  86262,     88,   4298,
         66416,  14767,   4994,   6457,   1174,    374,    264,  39747,   3560,
           571,     12,     31,   5737,   2835,   1847,   8040,    555,  80949,
           323,   7972,   5168,   1854,    369,    279,  32365,  42585,    662,
         45894,    304,   6186,    220,    679,     16,    304,   6457,   1174,
           433,    374,    279,   4948,   1847,    304,    279,  86262,     88,
          4298,   4101,    662,  21445,    287,    279,   1890,  37608,    315,
         39747,    323,   1972,    571, 

In [15]:
ex_t = ex_t.unsqueeze(0)
print(ex_t.shape)

torch.Size([1, 392])


In [16]:
#out = llama_1b(ex_t)

In [17]:
#out.logits.shape

In [18]:
#torch.softmax(out)

In [19]:
#logits = torch.softmax(out.logits, -1)
#tokens = torch.argmax(logits, -1)

In [20]:
#t_s = tokens.squeeze()


In [21]:
#t_s.shape

In [22]:
#out = tokenizer.decode(t_s)

In [23]:
out

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7597],
         [11.5920,  6.5140, 10.1322,  ..., -0.7728, -0.7726, -0.7735],
         [ 9.5127, 10.0392,  7.6556,  ...,  0.7077,  0.7083,  0.7083],
         ...,
         [ 6.2082,  5.8532,  6.3905,  ...,  0.0585,  0.0581,  0.0582],
         [ 9.4777, 11.9385, 10.1954,  ...,  2.2570,  2.2562,  2.2560],
         [ 7.1186,  8.8217,  9.4619,  ..., -0.7365, -0.7358, -0.7363]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[ 9.5610e-02,  1.7827e-01,  3.6755e-02,  ..., -1.7652e+00,
            1.7963e+00,  7.9892e-03],
          [ 1.2617e+00,  1.0782e+00, -1.2759e+00,  ...,  2.2001e+00,
           -2.0114e+00, -5.0596e-01],
          [-6.0104e+00, -1.4284e+00, -2.8390e+00,  ...,  8.7080e-01,
           -1.8995e+00, -2.1934e+00],
          ...,
          [ 4.4756e+00,  9.8246e-01, -2.3514e+00,  ...,  1.7411e+00,
           -1.6239e+00, -9.8035e-01],
          [ 2.466

In [48]:
out_3b = llama_3b(ex_t)
logits = torch.softmax(out_3b.logits, -1)
tokens = torch.argmax(logits, -1)
t_s = tokens.squeeze()
out = tokenizer.decode(t_s)
print(out)

Question yrie Chronicles 
The noou no Valkyria III3
 Lrecorded Chronicles
戦 )
 戦場のヴァルキュリア3 ～ Sen. "yria Chronicles the Battlefield 3 ) is also known to as Valkyria Chronicles III, of, is a tactical role-playing gameplaying- game game developed by Sega and published.Vision and the PlayStation Portable. It in Japan 2011 in Japan, it is the third game in the Valkyria Chronicles. Iting a same real of real role action-time-@ time gameplay as its predecessors, Valk game is parallel to the events game, is the eventsUnless Hero character a young battalion unit, under Empire of Gallia. the Second Europan War. are a missions ops. are leditted against the Empire Army known Theamy " ".The game was development in 2000, with on the number portion of the staff force on Valkyria Chronicles II. The the was the same  of the series, such also introduced a changes to including as the the game more accessible and new newcomers. The designs Fyoa Kazjō, composer Hitoshi Sakimoto returned returned to Valk gam

In [50]:
llama_3b.model.layers

ModuleList(
  (0-27): 28 x LlamaDecoderLayer(
    (self_attn): LlamaSdpaAttention(
      (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
      (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
      (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
      (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
      (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
      (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
    (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
  )
)

In [51]:
# check if intermediate output +can do backprop

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

llama_1b = AutoModelForCausalLM.from_pretrained('models/meta-llama/llama-3.2-1B')
llama_3b = AutoModelForCausalLM.from_pretrained('models/meta-llama/llama-3.2-3B')
tokenizer = AutoTokenizer.from_pretrained('tokenizers/meta-llama/llama-3.2-1B')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
llama_1b

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [3]:
llama_1b.model.layers[0]

LlamaDecoderLayer(
  (self_attn): LlamaSdpaAttention(
    (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear(in_features=2048, out_features=512, bias=False)
    (v_proj): Linear(in_features=2048, out_features=512, bias=False)
    (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)

In [4]:
a = llama_1b.model.layers[0]

In [5]:
dir(a)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_is_hf_initialized',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_no

In [6]:
import inspect

In [9]:
inspect.getsource(a)

TypeError: module, class, method, function, traceback, frame, or code object was expected, got LlamaDecoderLayer

In [10]:
inspect.getmodule(a)

<module 'transformers.models.llama.modeling_llama' from '/Users/sungwonkim/pycharm/MT Final/.venv/lib/python3.12/site-packages/transformers/models/llama/modeling_llama.py'>

In [11]:
getattr(a)

TypeError: getattr expected at least 2 arguments, got 1

In [1]:
import pickle

with open("data/chunked_wikitext2/train.pkl", "rb") as f:
    train_data = pickle.load(f)

In [2]:
lengths = [len(a) for a in train_data]
print(sum(lengths) / len(lengths))

17317.949125596184


In [3]:
chunk_size = 2000
new_chunks = []
for item in train_data:
    start_index = 0
    while start_index + chunk_size < len(item):
        new_chunks.append(item[start_index: start_index + chunk_size])
        start_index += chunk_size


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("tokenizers/meta-llama/llama-3.2-3B")

In [5]:
print(new_chunks[0])

 = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newc

In [6]:
tokenized_train = [tokenizer(_, return_tensors='pt') for _ in new_chunks]

In [7]:
tokenized_train[1]

{'input_ids': tensor([[128000,   3928,    323,   1935,    961,    304,  25664,   2403,   9354,
           8603,    662,  30129,    527,   3309,   1555,  20303,   2363,    571,
             12,     31,   1093,  21988,    449,  11625,   3752,  59234,   1174,
            449,   5885,  12365,  26310,   1555,  52468,   8982,  44783,    323,
          26310,   1555,    653,   3415,   7725,   1495,    662,    578,   2851,
          68711,   1555,    264,   4101,    315,  13790,  25664,   1174,  27115,
          36718,    439,  14370,    430,    649,    387,  26662,  48548,   1555,
            323,  35585,    291,    439,    814,    527,  36718,    662,    578,
           6149,    311,   1855,   3446,   3813,    389,    279,   2472,  35327,
          11911,    389,    459,   3927,   2851,    364,     82,   5603,    551,
            994,    832,   3072,    374,   4183,   1174,    279,   1023,    374,
          19584,   1022,    311,    279,   2851,    662,  42251,  25664,   1174,
            27

In [8]:
from transformers import AutoModelForCausalLM
llama_3b = AutoModelForCausalLM.from_pretrained("models/meta-llama/llama-3.2-3B")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
# example
a = llama_1b(input_ids = tokenized_train[1].input_ids, output_hidden_states=True)

In [12]:
detached = a.hidden_states[10].detach()

In [13]:
detached.squeeze()

tensor([[ 0.1367, -0.1160,  0.9267,  ...,  0.4339,  0.7350,  0.5163],
        [ 0.1470, -0.0941, -0.0739,  ..., -0.0782,  0.0868,  0.0040],
        [-0.1180, -0.0915,  0.2125,  ..., -0.2082, -0.0671, -0.1085],
        ...,
        [ 0.0608,  0.0664,  0.0936,  ..., -0.0209,  0.1380, -0.0480],
        [-0.0234, -0.0579,  0.1588,  ..., -0.1562, -0.0837, -0.0629],
        [-0.2190,  0.1680,  0.0766,  ..., -0.0076, -0.0299,  0.0545]])

In [14]:
from tqdm import tqdm

In [15]:
import torch

device = 'mps'
llama_1b.to(device)
intermediate_llama_1b_10 = []
#intermediate_llama_1b_5 = []

with torch.no_grad():
    for chunk in tqdm(tokenized_train):
        chunk = chunk.to(device)
        output = llama_1b(input_ids=chunk.input_ids, attention_mask=chunk.attention_mask, output_hidden_states=True)
        output_10 = output.hidden_states[10].detach().squeeze()
        output_5 = output.hidden_states[5].detach().squeeze()
        intermediate_llama_1b_10.append(output_10.cpu())
        #intermediate_llama_1b_5.append(output_5.cpu())
        del output, output_10, output_5
        torch.mps.empty_cache()  # Use torch.mps.empty_cache() if using MPS backend

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 5145/5145 [17:16<00:00,  4.97it/s]


In [16]:
#print(len(intermediate_llama_1b_5))
print(len(intermediate_llama_1b_10))

5145


In [17]:
# with open("tokenized/intermediate_llama_1b_5.pkl", 'wb') as f:
#     pickle.dump(intermediate_llama_1b_5, f)
with open("tokenized/intermediate_llama_1b_10.pkl", 'wb') as f:
    pickle.dump(intermediate_llama_1b_10, f)

In [11]:
from tqdm import tqdm
import torch

In [12]:
device = 'mps'
llama_3b.to(device)
intermediate_llama_3b_18 = []

with torch.no_grad():
    for chunk in tqdm(tokenized_train):
        chunk = chunk.to(device)
        output = llama_3b(input_ids=chunk.input_ids, attention_mask=chunk.attention_mask, output_hidden_states=True)
        output_18 = output.hidden_states[18].detach().squeeze()
        intermediate_llama_3b_18.append(output_18.cpu())
        del output, output_18
        torch.mps.empty_cache()

100%|██████████| 5145/5145 [38:48<00:00,  2.21it/s]


In [14]:
with open("tokenized/intermediate_llama_3b_18.pkl", 'wb') as f:
    pickle.dump(intermediate_llama_3b_18[:2000], f)

In [15]:
with open("tokenized/intermediate_llama_3b_18.pkl", 'ab') as f:
    pickle.dump(intermediate_llama_3b_18[2000:], f)